In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
pkdarabi_brain_tumor_image_dataset_semantic_segmentation_path = kagglehub.dataset_download('pkdarabi/brain-tumor-image-dataset-semantic-segmentation')

print('Data source import complete.')


In [ ]:
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
# loading any random image
train_path  = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/train/'
test_path = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/test/'
val_path  = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/valid/'

In [ ]:
train_images = [image for image in os.listdir(train_path) if image[-3:] =='jpg' ]
test_images = [image for image in os.listdir(test_path) if image[-3:] =='jpg' ]
val_images = [image for image in os.listdir(val_path) if image[-3:] =='jpg' ]
len(train_images),len(test_images),len(val_images)

In [ ]:
import glob
import os

train_annotations = glob.glob(os.path.join(train_path, '*.json'))
test_annotations = glob.glob(os.path.join(test_path, '*.json'))
val_annotations = glob.glob(os.path.join(val_path, '*.json'))


In [ ]:
import json

In [ ]:
train_annotations = json.load(open(train_annotations[0]))
test_annotations = json.load(open(test_annotations[0]))
val_annotations = json.load(open(val_annotations[0]))

In [ ]:
len(train_annotations['annotations']),len(test_annotations['annotations']),len(val_annotations['annotations']),

In [ ]:
import matplotlib.pyplot as plt
def visualize_random_images(n=5):
  # select n random images
  # use cv and plt to show these images
  indices = np.random.randint(0, len(train_annotations['images']), size=n)

  images =[train_annotations['images'][i] for i in indices ]

  annotations = [train_annotations['annotations'][i] for i in indices ]
  j=1
  plt.figure(figsize=(12, 4 * 2 * n))
  for img,ann in zip(images,annotations):
    plt.subplot(n,3,j)
    j+=1
    image = cv2.imread(train_path + img['file_name'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    # create masked images from annotations
    segmentation = ann['segmentation']
    segmentation = np.array(segmentation[0], dtype=np.int32).reshape(-1, 2)

    cv2.polylines(image, [segmentation], isClosed=True, color=(0, 255, 0), thickness=2)  # Green color with thickness 2

    plt.subplot(n,3,j)

    plt.imshow(image)
    j+=1
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    cv2.fillPoly(mask, [segmentation], color=1)
    plt.subplot(n,3,j)

    plt.imshow(mask,cmap='gray')
    j+=1

visualize_random_images()

# Create binary mask for all images

In [ ]:
def _train_masks():
    print('train masks')
    mask_dir = '/kaggle/working/train_masks/'
    os.makedirs(mask_dir, exist_ok=True)
    totalImages = len(train_annotations['images'])
    done = 0
    for img,ann in zip(train_annotations['images'],train_annotations['annotations']):
        path = train_path + img['file_name']
        mask_path = mask_dir + img['file_name']
        # load image in open cv
        image = cv2.imread(path)
        segmentation = ann['segmentation']
        segmentation = np.array(segmentation[0], dtype=np.int32).reshape(-1, 2)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        cv2.fillPoly(mask, [segmentation], color=(255,255,255))
        cv2.imwrite(mask_path, mask)
        done+=1
        print(f"train  {done} / {totalImages} ")

def _test_masks():
    print('test masks')

    totalImages = len(test_annotations['images'])
    done = 0
    mask_dir = '/kaggle/working/test_masks/'
    os.makedirs(mask_dir, exist_ok=True)

    for img,ann in zip(test_annotations['images'],test_annotations['annotations']):
        path = test_path + img['file_name']
        mask_path = mask_dir + img['file_name']
        # load image in open cv
        image = cv2.imread(path)
        segmentation = ann['segmentation']
        segmentation = np.array(segmentation[0], dtype=np.int32).reshape(-1, 2)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        cv2.fillPoly(mask, [segmentation], color=(255,255,255))
        cv2.imwrite(mask_path, mask)
        done+=1

        print(f"test  {done} / {totalImages} ")


def _val_masks():
    print('val masks')
    totalImages = len(val_annotations['images'])
    done = 0
    mask_dir = '/kaggle/working/val_masks/'
    os.makedirs(mask_dir, exist_ok=True)

    for img,ann in zip(val_annotations['images'],val_annotations['annotations']):
        path = val_path + img['file_name']
        mask_path = mask_dir + img['file_name']
        # load image in open cv
        image = cv2.imread(path)
        segmentation = ann['segmentation']
        segmentation = np.array(segmentation[0], dtype=np.int32).reshape(-1, 2)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        cv2.fillPoly(mask, [segmentation], color=(255,255,255))
        cv2.imwrite(mask_path, mask)
        done+=1
        print(f"val  {done} / {totalImages} ")


from threading import Thread
def make_masks():
  threads = []
  threads.append( Thread(target=_train_masks))

  threads.append( Thread(target=_test_masks))

  threads.append( Thread(target=_val_masks))
  for t in threads:
    t.start()
  for t in threads:
    t.join()
  print('complete')
  return

make_masks()


# load and split data

In [ ]:
def load_data():
    target_size = (512, 512)
    train_mask_dir = '/kaggle/working/train_masks/'
    X_train =  [cv2.resize(cv2.imread(train_path + image['file_name']),target_size) for image in train_annotations['images']]
    y_train = [cv2.resize(cv2.imread(train_mask_dir + image['file_name'],cv2.IMREAD_GRAYSCALE),target_size ) for image in train_annotations['images']]
    X_train = np.array(X_train)
    y_train = np.expand_dims(np.array(y_train), axis=-1)

    X_train = X_train.astype('float32') / 255.0
    y_train = y_train.astype('float32') / 255.0
    y_train = (y_train > 0.5).astype(np.float32)





    val_mask_dir = '/kaggle/working/val_masks/'

    X_val =  [cv2.resize(cv2.imread(val_path + image['file_name']),target_size) for image in val_annotations['images']]
    y_val = [cv2.resize(cv2.imread(val_mask_dir + image['file_name'],cv2.IMREAD_GRAYSCALE),target_size) for image in val_annotations['images']]
    X_val = np.array(X_val)
    y_val = np.expand_dims(np.array(y_val), axis=-1)



    X_val = X_val.astype('float32') / 255.0
    y_val = y_val.astype('float32') / 255.0
    y_val = (y_val > 0.5).astype(np.float32)



    return X_train,y_train,X_val,y_val

def load_test_data():
    target_size = (512, 512)

    test_mask_dir = '/kaggle/working/test_masks/'
    X_test =  [cv2.resize(cv2.imread(test_path + image['file_name']),target_size) for image in test_annotations['images']]
    y_test = [cv2.resize(cv2.imread(test_mask_dir + image['file_name'],cv2.IMREAD_GRAYSCALE),target_size) for image in test_annotations['images']]
    X_test = np.array(X_test)
    y_test = np.expand_dims(np.array(y_test), axis=-1)


    X_test = X_test.astype('float32') / 255.0
    y_test = y_test.astype('float32') / 255.0
    y_test = (y_test > 0.5).astype(np.float32)
    return X_test,y_test


In [ ]:
X_train,y_train,X_val,y_val = load_data()


In [ ]:
X_train.shape , y_train.shape  ,X_val.shape ,y_val.shape

# define Unet model

In [ ]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred, smooth=1e-7):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coefficient(y_true, y_pred)
    return loss



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def convBlock(inputs,filters):
    c1 = layers.Conv2D(filters, kernel_size = (3,3), padding = "same")(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.ReLU()(c1)

    # Taking first input and implementing the second conv block
    c2 = layers.Conv2D(filters, kernel_size = (3,3), padding = "same")(c1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.ReLU()(c2)

    return c2


def encoderBlock(inputs,filters):
    c = convBlock(inputs, filters)
    m = layers.MaxPooling2D(strides = (2,2))(c)
    return c, m


def decoderBlock(inputs, skip, filters=64):
    u = layers.Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(inputs)
    skip = layers.Concatenate()([u, skip])
    out = convBlock(skip, filters)
    return out


def uNet(image_size):
    inputs = layers.Input(image_size)

    # Construct the encoder blocks and increasing the filters by a factor of 2
    skip1, encoder_1 = encoderBlock(inputs, 64)
    skip2, encoder_2 = encoderBlock(encoder_1, 128)
    skip3, encoder_3 = encoderBlock(encoder_2, 256)
    skip4, encoder_4 = encoderBlock(encoder_3, 512)

    # Preparing the next block
    conv_block = convBlock(encoder_4, 1024)

    # Construct the decoder blocks and decreasing the filters by a factor of 2
    decoder_1 = decoderBlock(conv_block, skip4, 512)
    decoder_2 = decoderBlock(decoder_1, skip3, 256)
    decoder_3 = decoderBlock(decoder_2, skip2, 128)
    decoder_4 = decoderBlock(decoder_3, skip1, 64)

    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(decoder_4)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
def test_accuracy(model):
    X_test,y_test = load_test_data()
    batch_size = 2
    predictions = model.predict(X_test, batch_size=batch_size)

    # Reshape predictions if necessary (assuming single-channel masks)
    y_pred = np.squeeze(predictions)
    y_pred = np.expand_dims(y_pred, axis=-1)

    threshold = 0.5
    y_pred_binary = (y_pred >= threshold).astype(int)

    # Calculate pixel-wise accuracy
    accuracy = np.mean(y_pred_binary == y_test)

    dice_coefficient_score = dice_coefficient(y_test, y_pred)
    dice_loss_score = dice_loss(y_test, y_pred)
    print(f"Pixel-wise Accuracy: {accuracy:.4f}")
    print(f'Dice Coefficient: {dice_coefficient_score:.4f}')
    print(f'Dice Loss: {dice_loss_score:.4f}')
    X_test,y_test = 0,0

# Initialize Model

In [ ]:
input_shape = (512,512,3)
model = uNet(input_shape)
model.summary()
model.load_weights('/kaggle/input/brain_unet/keras/default/1/brain_tumor_unet.weights.h5')

In [ ]:
initial_lr = 1e-4
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate = initial_lr),loss = dice_loss,metrics=['accuracy',dice_coefficient])

## Saved Weights accuracy


In [ ]:
test_accuracy(model)

In [ ]:
import cv2
import numpy as np

def predict(image_path, model, threshold=0.5):
    # Read and preprocess the image
    image = cv2.imread(image_path)
    print("Original Image Shape:", image.shape)

    # Resize the image to match model's expected sizing
    resized_image = cv2.resize(image, (512, 512))
    print("Resized Image Shape:", resized_image.shape)

    # Expand dimensions to match the batch size used by the model
    input_image = np.expand_dims(resized_image, axis=0)

    # Preprocess input (normalize to [0, 1] range)
    input_image = input_image.astype('float32') / 255.0

    # Perform prediction
    pred_mask = model.predict(input_image)

    # Apply threshold to prediction mask
    pred_mask[pred_mask >= threshold] = 1  # set values >= threshold to 1 (foreground)
    pred_mask[pred_mask < threshold] = 0   # set values < threshold to 0 (background)

    # If your model outputs probabilities, you might need to squeeze the mask
    pred_mask = np.squeeze(pred_mask, axis=0)

    # Count number of pixels where mask is 1 (foreground)
    num_pixels = np.count_nonzero(pred_mask)
    print("shape mask",pred_mask.shape)
    return pred_mask

# Example usage:
# Assuming 'model' is your trained model and 'image_path' is the path to your image
# pred_mask, num_pixels = predict(image_path, model)
# print("Number of foreground pixels:", num_pixels)


In [ ]:
import matplotlib.pyplot as plt
def test_random_images(n=5):
  # select n random images
  # use cv and plt to show these images
    test_mask_dir = '/kaggle/working/test_masks/'

    indices = np.random.randint(0, len(test_annotations['images']), size=n)

    images =[test_annotations['images'][i] for i in indices ]

    annotations = [test_annotations['annotations'][i] for i in indices ]
    j=1
    plt.figure(figsize=(12, 4 * 2 * n))
    for img,ann in zip(images,annotations):
        plt.subplot(n,3,j)
        j+=1
        image = cv2.imread(test_path + img['file_name'])
        plt.imshow(image)
        plt.title("Input")
        mask = predict(test_path + img['file_name'],model)
#         print(mask.shape)

        result_image = np.zeros((512, 512,1), dtype=np.uint8)

        result_image[mask == 1] = 255

        true_mask = cv2.imread(test_mask_dir + img['file_name'])
        plt.subplot(n,3,j)
        j+=1
        plt.imshow(true_mask)
        plt.title("true_mask")

        plt.subplot(n,3,j)
        j+=1
        plt.imshow(result_image,cmap='gray')
        plt.title("Prediction")





test_random_images()

# Retraining the model

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.5,
                                   patience=5,
                                   verbose=1,
                                   mode='min',
                                   cooldown=3,
                                   min_lr=1e-10)


callbacks_list = [ reduceLROnPlat]

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


# Now fit your model
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=2,
    callbacks=callbacks_list
)

In [ ]:
model.save_weights('brain_tumor_unet.weights.h5')



## Retrained Weights Metrics

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the History object returned by model.fit()
def plot_training_history(history):
    # Extract metrics from the history object
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    dice = history.history['dice_coefficient']
    val_dice = history.history['val_dice_coefficient']

    epochs = range(1, len(loss) + 1)

    # Plot Loss
    plt.figure(figsize=(14, 4))
    plt.subplot(1, 3, 1)
    plt.plot(epochs, loss,  label='Training loss')
    plt.plot(epochs, val_loss,  label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 3, 2)
    plt.plot(epochs, accuracy, label='Training accuracy')
    plt.plot(epochs, val_accuracy, label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Dice Coefficient
    plt.subplot(1, 3, 3)
    plt.plot(epochs, dice,  label='Training Dice')
    plt.plot(epochs, val_dice, label='Validation Dice')
    plt.title('Training and Validation Dice Coefficient')
    plt.xlabel('Epochs')
    plt.ylabel('Dice Coefficient')
    plt.legend()

    plt.tight_layout()
    plt.show()




plot_training_history(history)

In [ ]:
test_random_images()

# Test Model

In [ ]:
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os



In [ ]:
test_path = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/test/'

In [ ]:
import glob
import os
import json
test_annotations = glob.glob(os.path.join(test_path, '*.json'))
test_annotations = json.load(open(test_annotations[0]))
len(test_annotations['annotations'])

## Get Metrics

In [ ]:
test_accuracy(model)